In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## User Inputs

In [ ]:
infile_test = 'vector_conversion_dataset_test.npz'
infile_train = 'vector_conversion_dataset.npz'
numtrials = 5
numfuncs = 3

## Loading Data and Model

In [ ]:
npfile_test = np.load(infile_test)
inputs_test = npfile_test['inputs']
targets_test = npfile_test['outputs']

npfile_train = np.load(infile_train)
inputs_test = npfile_train['inputs']
targets_test = npfile_train['outputs']

In [ ]:
func = 0
trial = 0

for func in range(1,numfuncs+1):
    for trial in range(1,numtrials+1):

        model_path = 'v_conv_model_10node_fn{}_{}'.format(func,trial)
        model = torch.load(model_path)

        # standardize inputs
        inputMeans = inputs_train[0:int(inputs_train.shape[0]),:].mean(axis=0)
        inputStdDevs = inputs_train[0:int(inputs_train.shape[0]),:].std(axis=0)
        inputs = (inputs_test-inputMeans)/inputStdDevs
        inputs = torch.from_numpy(inputs).float()

        # get model outputs
        output = model(inputs)
        output = output.data.numpy()

        # unstandardize outputs
        outputMeans = targets_train[0:int(targets_train.shape[0]),:].mean(axis=0) 
        outputStdDevs = targets_train[0:int(targets_train.shape[0]),:].std(axis=0)

        output = output*outputStdDevs+outputMeans

        # calculate differences for every element
        diff = targets_test-output

        # set maxes, mins
        minDiff = np.amin(diff,axis=0)
        maxDiff = np.amax(diff,axis=0)
        stdDiff = np.std(diff,axis=0)
        meanDiff = np.mean(diff,axis=0)
        minTarget = np.amin(target,axis=0)
        maxTarget = np.amax(target,axis=0)
        minOutput = np.amin(output,axis=0)
        maxOutput = np.amax(output,axis=0)
        minInput = np.amin(inputs,axis=0)
        maxInput = np.amax(inputs,axis=0)

        ## Generating Plots

        # create a histogram for each output in the target

        for itarg in range(target.shape[1]):

            diffarray = diff[:,itarg]
            targetarray = targets_test[:,itarg]
            outputarray = output[:,itarg]
            inputsarray = inputs_test[:,itarg]
    
            fig1 = plt.figure()
            plt.hist(diffarray, bins=100)
            plt.title('Target - Output (index:{})'.format(itarg))
            plt.xlim(meanDiff[itarg]-3*stdDiff[itarg],meanDiff[itarg]+3*stdDiff[itarg])
            plt.figure(figsize=(20,15))
            fig1.savefig('./histograms/v_conv_10nodes/fn{}/test{}/hist_1_{}.png'.format(func,trial,itarg), format='png')
    
            fig2 = plt.figure()
            plt.hist2d(targetarray,outputarray, bins=400)
            plt.title('Output vs Target (index:{})'.format(itarg))
            plt.xlim(minTarget[itarg],maxTarget[itarg])
            plt.ylim(minOutput[itarg],maxOutput[itarg])
            plt.figure(figsize=(20,15))
            fig2.savefig('./histograms/v_conv_10nodes/fn{}/test{}/hist_2_{}.png'.format(func,trial,itarg), format='png')
            
            fig3 = plt.figure()
            sns.regplot(x=targetarray, y=diffarray, x_bins=400, fit_reg=None)
            plt.title('Prof(Output) vs Target (index:{})'.format(itarg))
            plt.xlim(minTarget[itarg],maxTarget[itarg])
            plt.figure(figsize=(20,15))
            fig3.savefig('./histograms/v_conv_10nodes/fn{}/test{}/hist_3_{}.png'.format(func,trial,itarg), format='png')
            
            fig4 = plt.figure()
            plt.hist2d(inputsarray,outputarray, bins=400)
            plt.title('O_{{{0}}} vs I_{{{0}}}'.format(itarg))
            plt.xlim(minOutput[itarg],maxOutput[itarg])
            plt.ylim(minInput[itarg],maxInput[itarg])
            plt.figure(figsize=(20,15))
            fig4.savefig('./histograms/v_conv_10nodes/fn{}/test{}/hist_4_{}.png'.format(func,trial,itarg), format='png')
            
            fig5 = plt.figure()
            sns.regplot(x=inputsarray, y=diffarray, x_bins=400, fit_reg=None)
            plt.title('Prof(Output) vs Input (index:{})'.format(itarg))
            plt.xlim(minInput[itarg],maxInput[itarg])
            plt.figure(figsize=(20,15))
            fig5.savefig('./histograms/v_conv_10nodes/fn{}/test{}/hist_5_{}.png'.format(func,trial,itarg), format='png')             
            